# Clustering Distance Metrics

## Functions

Distance Metric: Variation of Information

In [ ]:
import numpy as np
import scipy.stats as ss
from sklearn.metrics import mutual_info_score

def numBins(nObs,corr=None):
    # Optimal number of bins for discretization
    if corr is None: # univariate case
        z=(8+324*nObs+12*(36*nObs+729*nObs**2)**.5)**(1/3.)
        b=round(z/6.+2./(3*z)+1./3)
    else: # bivariate case
        b=round(2**-.5*(1+(1+24*nObs/(1.-corr**2))**.5)**.5)
    return int(b)

def varInfo(x,y,norm=False):
    # variation of information
    bXY=numBins(x.shape[0],corr=np.corrcoef(x,y)[0,1])
    
    cXY=np.histogram2d(x,y,bXY)[0]
    iXY=mutual_info_score(None,None,contingency=cXY)
    hX=ss.entropy(np.histogram(x,bXY)[0]) # marginal
    hY=ss.entropy(np.histogram(y,bXY)[0]) # marginal
    vXY=hX+hY-2*iXY # variation of information
    if norm:
        hXY=hX+hY-iXY # joint
        vXY/=hXY # normalized variation of information
    return vXY

Cluster Algos

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples

def clusterKMeansBase(corr0,maxNumClusters=10,n_init=10):

    x, silh = ((1 - corr0.fillna(0)) / 2.) ** .5, pd.Series(dtype=np.float64) # eval observations matrix from corr0

    # try different initializations
    for init in range(n_init):
        
         # try different cluster numbers
        for i in range(2,maxNumClusters+1):

            # perform kmeans
            kmeans_ = KMeans(n_clusters=i, n_init=1)
            kmeans_ = kmeans_.fit(x)

            # compute silhouette score and quality measure
            silh_= silhouette_samples(x,kmeans_.labels_)
            stat = (silh_.mean() / silh_.std(), silh.mean() / silh.std())
            
            # keep fit if previous stat was better
            if np.isnan(stat[1]) or stat[0]>stat[1]:
                silh, kmeans = silh_, kmeans_
    
    # order results
    newIdx=np.argsort(kmeans.labels_)
    corr1=corr0.iloc[newIdx] # reorder rows
    corr1=corr1.iloc[:,newIdx] # reorder columns

    # rename clusters
    clstrs = {i:corr0.columns[np.where(kmeans.labels_==i)[0]].tolist() for i in np.unique(kmeans.labels_)} # cluster members
    silh = pd.Series(silh, index=x.index)
    
    return corr1, clstrs, silh

In [ ]:
from sklearn.metrics import silhouette_samples

def makeNewOutputs(corr0,clstrs,clstrs2):
    clstrsNew={}

    # build lists for clusters to compare
    for i in clstrs.keys():
        clstrsNew[len(clstrsNew.keys())]=list(clstrs[i])
    for i in clstrs2.keys():
        clstrsNew[len(clstrsNew.keys())]=list(clstrs2[i])

    # build new correlation matrix from clstrsNew
    newIdx = [j for i in clstrsNew for j in clstrsNew[i]]
    corrNew = corr0.loc[newIdx,newIdx]

    # compute distance matrix of new correlation matrix
    x = ((1 - corr0.fillna(0)) / 2.) ** .5

    # make array for new cluster labels
    kmeans_labels = np.zeros(len(x.columns))

    # assign labels of x to labels of kmeans_labels
    for i in clstrsNew.keys():
        idxs = [x.index.get_loc(k) for k in clstrsNew[i]]
        kmeans_labels[idxs] = i

    # compute silhouette scores within x using kmeans_labels
    silhNew = pd.Series(silhouette_samples(x,kmeans_labels), index=x.index)

    return corrNew, clstrsNew, silhNew

In [ ]:
def clusterKMeansTop(corr0,maxNumClusters=None,n_init=10):

    if maxNumClusters == None: 
        maxNumClusters = corr0.shape[1] - 1
    
    # run base clustering on corr0
    corr1, clstrs, silh = clusterKMeansBase(corr0, maxNumClusters=min(maxNumClusters, corr0.shape[1]-1), n_init=n_init)

    # get quality score of each cluster from base clustering
    clusterTstats = {i:np.mean(silh[clstrs[i]]) / np.std(silh[clstrs[i]]) for i in clstrs.keys()}

    # compute mean of quality scores
    tStatMean = sum(clusterTstats.values()) / len(clusterTstats)
    
    # find subset of clusters with quality score less than mean
    redoClusters = [i for i in clusterTstats.keys() if clusterTstats[i] < tStatMean]
    
    if len(redoClusters) <= 1:
        return corr1, clstrs, silh # no clusters to redo, return previous base clustering results
    else:
        # build new correlation matrix from clusters to redo
        keysRedo = [j for i in redoClusters for j in clstrs[i]]
        corrTmp = corr0.loc[keysRedo,keysRedo]

        # get stats of actual clusters to redo
        tStatMean = np.mean([clusterTstats[i] for i in redoClusters])

        # run top clustering on new correlation matrix (recursive call)
        corr2, clstrs2, silh2 = clusterKMeansTop(corrTmp, maxNumClusters=min(maxNumClusters, corrTmp.shape[1]-1), n_init=n_init)
    
        # Make new outputs, if necessary
        corrNew,clstrsNew,silhNew = makeNewOutputs(corr0, {i:clstrs[i] for i in clstrs.keys() if i not in redoClusters}, clstrs2)
        
        # get new quality scores for redone clusters
        newTstatMean = np.mean([np.mean(silhNew[clstrsNew[i]]) / np.std(silhNew[clstrsNew[i]]) for i in clstrsNew.keys()])

        if newTstatMean <= tStatMean:
            return corr1, clstrs, silh # return previous base clustering results if quality score is worse
        else:
            return corrNew, clstrsNew, silhNew # return new clustering results if quality score is better

## Build synthetic test data

In [ ]:
from sklearn.datasets import make_classiﬁcation

def getTestData(n_features=100 ,n_informative=25, n_redundant=25, n_samples=10000, random_state=0, sigmaStd=.0):
    
    # generate a random dataset for a classiﬁcation problem
    np.random.seed(random_state)
    X,y = make_classiﬁcation(n_samples=n_samples, n_features=n_features-n_redundant, n_informative=n_informative, n_redundant=0, shufﬂe=False, random_state=random_state)
    
    # name the columns
    cols = ['I' + str(i) for i in range(n_informative)] # I = Informative
    cols += ['N' + str(i) for i in range(n_features - n_informative - n_redundant)] # N = Noise
    
    # make dataframe
    X,y = pd.DataFrame(X, columns=cols),pd.Series(y)
    
    # choose random informative features to make redundant
    i = np.random.choice(range(n_informative), size=n_redundant)

    # make inverted dict to print
    r_to_i_map = {f"R{k}":f"I{v}" for k,v in enumerate(i)}  # Ri : Ii
    i_to_r_map = {}
    for k,v in r_to_i_map.items():
        i_to_r_map[v] = i_to_r_map.get(v, []) + [k] # Ii : Ri
    print(f"Informative Features used to generate Redundant Features: ")
    for k,v in i_to_r_map.items():
        print(f"{k} : {v}")

    # make redundant features
    for k,j in enumerate(i):
        X['R' + str(k)] = X['I' + str(j)] + np.random.normal(size = X.shape[0]) * sigmaStd # R = Redundant

    return X,y

In [ ]:
X,y = getTestData(40,5,30,10000,sigmaStd=.1)

## Compute Distance Metric

In [ ]:
l = len(X.columns)
metric = np.full([l,l], np.nan)
for i in range(l):
    for j in range(l):
        if not i == j: 
            metric[i,j] = varInfo(X.iloc[:,i].values, X.iloc[:,j].values, norm=True)
        else:
            metric[i,j] = 0

corr0 = pd.DataFrame(metric, index=X.columns, columns=X.columns)

In [ ]:
import plotly.graph_objects as go


def plot_corr(corr0):
    fig = go.Figure(
        data=go.Heatmap(
            z=corr0,
            x=corr0.index.astype(str),
            y=corr0.columns.astype(str),
            colorscale='Viridis'
            )
        )

    fig.update_layout(
        title='VarInfo Matrix',
        width=800,
        height=800,)
    
    fig.show()

In [ ]:
plot_corr(corr0)

## Clustering Distance Metric

In [ ]:
corr1, clstrs, silh = clusterKMeansTop(corr0=corr0, maxNumClusters=10, n_init=10)

In [ ]:
plot_corr(corr1)

In [ ]:
print("Cluster : Features")
for k,v in clstrs.items():
    print(k,":", v)